In [18]:
# downloading the metadata file if it does not exist
import os
if not os.path.isfile("ladi_images_metadata.csv"):
    !wget http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_images_metadata.csv

In [15]:
# importing the usual things
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json
import shutil

# for downloading images
import requests

# importing the geoprocessing libraries
import geopandas as gpd
import fiona # library for reading/writing GIS files, comes w/ geopandas
from shapely.geometry import Point, LineString, Polygon

In [7]:
# change parameters for individual projects
target_date = "2017-10-13"
project_path = "projects/mercedita/"

In [8]:
# load the csv as a gdf
df = pd.read_csv("ladi_images_metadata.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)
df = df.loc[target_date]

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.gps_lon, df.gps_lat))
gdf.crs = 'epsg:4326'

geojson = gpd.read_file(project_path + 'map.geojson')
poly = geojson.iloc[0, 0]

In [9]:
sample = gdf[gdf.geometry.apply(lambda x: x.within(poly))]
print(sample)

                                                         uuid    gps_lat  \
timestamp                                                                  
2017-10-13 13:05:39  2503270a13156b960939ad423d4388ad53af7333  18.005617   
2017-10-13 13:05:42  2f956e99ade5395c494da3c1b5b8a175bcdd0e3f  18.005430   
2017-10-13 13:05:44  15fc4f2142c4355aef9bfbedbfdd9e99aa068e46  18.005310   
2017-10-13 13:05:46  fce709f4b8f07f2d82377beac75665ef8a9a64de  18.005193   
2017-10-13 13:05:49  bc865150138500e26a07f79db79c846a54b0c133  18.005002   
...                                                       ...        ...   
2017-10-13 13:16:27  29646799f2905d8bf2674cbf5a139064f9ee8e36  18.016165   
2017-10-13 13:16:29  ce45a0b24b9ac28e525092ef3d1d10e4f001264d  18.016827   
2017-10-13 13:19:09  0b4a7d3a217b2de3e77465903b3864c055b66ee6  17.997250   
2017-10-13 13:19:10  67ebfb8dc25162ef12997cf2fcc8b53c05246638  17.996700   
2017-10-13 16:37:43  e24b066305a3e3c2c36739c35672b5e8501ea7c8  18.009555   

           

In [24]:
def create_project(sample, project_path):
    # copying the config file
    try:
        shutil.copy("config.yaml", project_path)
    except IOError as e:
        print("Unable to copy file. %s" % e)
    except:
        print("Unexpected error:", sys.exc_info())
    
    # get the list of urls to download
    url_list = sample["url"].to_list()
    
    # create the images folder if it does not currently exist
    image_folder_path = project_path + "images/"
    if not os.path.exists(image_folder_path):
        os.mkdir(image_folder_path)
        
    # iterate over the images, download them all
    for url in url_list:
        image_name = url.split("/")[-1]
        try:
            r = requests.get(url)
        except:
            continue
        image_path = image_folder_path + image_name
        with open(image_path, 'wb') as f:
            f.write(r.content)

In [25]:
create_project(sample, project_path)

ConnectionError: HTTPSConnectionPool(host='ladi.s3-us-west-2.amazonaws.com', port=443): Max retries exceeded with url: /Images/FEMA_CAP/9075/614109/DSC_0030_bc0af34a-6f95-416d-b9ff-d76ee2132e6a.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa608050fd0>: Failed to establish a new connection: [Errno 110] Connection timed out',))